# Chapter 12

- Tensorflow Python API

| Purpose | Tensorflow API |
| :------ | :------------- |
| High-level API | `tf.keras`, `tf.estimator` | 
| Low-level API | `tf.nn`, `tf.losses`, `tf.metrics`, `tf.optimizers`, `tf.train`, `tf.initializers` |
| Autodiff | `tf.GradientTape`, `tf.gradients()` |
| I/O & Preprocessing | `tf.data`, `tf.feature_column`, `tf.audio`, `tf.image`, `tf.io`, `tf.queue` |
| Tensorboard | `tf.summary` |
| Deployment and optimization | `tf.distribute`, `tf.saved_model`, `tf.autograph`, `tf.graph_util`, `tf.lite`, `tf.quantization`, `tf.tpu`, `tf.xla` |
| Special data structures | `tf.lookup`, `tf.nest`, `tf.ragged`, `tf.sets`, `tf.sparse`, `tf.strings` |
| Math | `tf.math`, `tf.linalg`, `tf.signal`, `tf.random`, `tf.bitwise` |
| Miscellaneous | `tf.compat`, `tf.config` |

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Tensor data structures and operations

1. Data structure

|  Datatype| description |
|:----|:----|
| `tf.constant()` | Immutable tensor |
| `tf.Variable()` | Mutable tensor. Use `assign()` and related methods, and `scatter_nd_update()` to update value |
| `tf.SparseTensor` | Represents tensors with mostly zeros | 
| `tf.TensorArray` | Represents list with tensors. All tensors must have the same shape and dtype. Fixed size by default but can be made mutable | 
| `tf.RaggedTensor` | Represents list of lists of tensors, where every tensor has the same shape and dtype | 
| string tensors (`tf.string`) | Represents string data (byte strings). `tf.strings` package contains ops for byte and unicode strings. `tf.string` is atomic (the length does not appear in tensor's shape, and will only be made visible after converting to `tf.int32` unicode representation tensor). See example. | 
| Sets | Defined as regular tensors |
| Queues | Store tensors across multiple steps, includes: `PriorityQueue`, `RandomShuffleQueue`, `PaddingFIFOQueue` | 


2. Operations

|  Operation| description |
|:----|:----|
| + | addition (calls `tf.__add__`) |
| - | subtraction (calls `tf.__sub__`) |
| * | (component wise) multiplication  (calls `tf.__mul__`) |
| @ | matrix multiplication (calls `tf.__matmul__`) |
| / | division (calls `tf.__div__`) |
| `transpose()` | Transpose a matrix | 


In [18]:
# define scaler
print(tf.constant(42))

# define matrix
print("\n", tf.constant([[1,2,3], [4,5,6]]))

# shape and dtype 
t = tf.constant([[1,2,3], [4,5,6]])
print("\n", (t.shape, t.dtype))

# tensor indexing
print("\n", t[:, 1], "\n" ,t[:, 1, tf.newaxis])

tf.Tensor(42, shape=(), dtype=int32)

 tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)

 (TensorShape([2, 3]), tf.int32)

 tf.Tensor([2 5], shape=(2,), dtype=int32) 
 tf.Tensor(
[[2]
 [5]], shape=(2, 1), dtype=int32)


In [20]:
(t + 10, tf.square(t), t @ tf.transpose(t))

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[11, 12, 13],
        [14, 15, 16]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[ 1,  4,  9],
        [16, 25, 36]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[14, 32],
        [32, 77]], dtype=int32)>)

`tf.transpose()` is different from numpy `.T` attribute: `tf` creates a new tensor with the transposed data, while `numpy` `.T` attribute is only a **view** of the transposed data. 

`keras.backend` provides interface to call corresponding tensorflow ops. 

In [24]:
# from numpy array to tf tensor
a = np.array([1,2,3])
print(tf.constant(a))

# from tf tensor to numpy array
print(t.numpy())

tf.Tensor([1 2 3], shape=(3,), dtype=int64)
[[1 2 3]
 [4 5 6]]


In [8]:
# type conversion
# tf does not allow implicit type conversions. Type conversion should be done
# explicitly through tf.cast() 

try:
     tf.constant([42.]) + tf.constant([42])
except Exception as e:
     print(e)

# Use tf.cast() to cast datatypes
t1 = tf.constant([42], dtype=tf.float64)
t2 = tf.constant([42], dtype=tf.int16)
tf.cast(t1, tf.int16) + t2

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]


<tf.Tensor: shape=(1,), dtype=int16, numpy=array([84], dtype=int16)>

In [21]:
# tf.constant() --> immutable
# tf.variable() --> mutable, suitable for weights and optims

g1 = tf.Variable([[1,2,3], [4,5,6]])
g2 = tf.Variable([[1,2,3], [4,5,6]])
g3 = tf.Variable([[1,2,3], [4,5,6]])
print(
    g1.assign(2 * g1), # assign the operation of * 2 to g
    g2[:, 0].assign(42), # assign value 42 to selected index
    g3.scatter_nd_update(indices = [[0,0], [1,2]],
                         updates = [100, 200]) # use scatter_nd_update to update variable
)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2,  4,  6],
       [ 8, 10, 12]], dtype=int32)> <tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[42,  2,  3],
       [42,  5,  6]], dtype=int32)> <tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[100,   2,   3],
       [  4,   5, 200]], dtype=int32)>


In [14]:
# string

(tf.constant("cafè"),
 tf.constant(b"tensorflow"), # b for byte string. ASCII literals only
 tf.constant([ord(c) for c in "cafè"])) # Unicode point encoding

(<tf.Tensor: shape=(), dtype=string, numpy=b'caf\xc3\xa8'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'tensorflow'>,
 <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 232], dtype=int32)>)

In [13]:
u = tf.constant([ord(c) for c in "cafè"])
uu = tf.strings.unicode_encode(tf.constant(u), "UTF-8")

(tf.strings.length(uu, unit="UTF8_CHAR"), tf.strings.unicode_decode(uu, "UTF-8"))

(<tf.Tensor: shape=(), dtype=int32, numpy=4>,
 <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 232], dtype=int32)>)

In [8]:
# string arrays 
p = tf.constant(["Café", "Coffee", "caffè", "咖啡"])
print(tf.strings.length(p, unit = "UTF8_CHAR"))

tf.Tensor([4 6 5 2], shape=(4,), dtype=int32)


In [12]:
# ragged tensor
r = tf.strings.unicode_decode(p, "UTF-8")
print(r, r[1], r[1:3]) 

<tf.RaggedTensor [[67, 97, 102, 233], [67, 111, 102, 102, 101, 101],
 [99, 97, 102, 102, 232], [21654, 21857]]> tf.Tensor([ 67 111 102 102 101 101], shape=(6,), dtype=int32) <tf.RaggedTensor [[67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232]]>


In [18]:
r1 = tf.ragged.constant([[65, 66], [], [67]])
r2 = tf.ragged.constant([[68, 69, 79], [71], [], [72, 83]])
(tf.concat([r, r1], axis = 0), # concatenate two ragged tensors
 tf.concat([r, r2], axis = 1)) 

(<tf.RaggedTensor [[67, 97, 102, 233], [67, 111, 102, 102, 101, 101],
  [99, 97, 102, 102, 232], [21654, 21857], [65, 66], [], [67]]>,
 <tf.RaggedTensor [[67, 97, 102, 233, 68, 69, 79], [67, 111, 102, 102, 101, 101, 71],
  [99, 97, 102, 102, 232], [21654, 21857, 72, 83]]>)

In [26]:
tf.strings.unicode_encode(r1, "UTF-8")

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'AB', b'', b'C'], dtype=object)>

In [20]:
r2.to_tensor() # convert back to tensor

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[68, 69, 79],
       [71,  0,  0],
       [ 0,  0,  0],
       [72, 83,  0]], dtype=int32)>

In [3]:
# sparse tensor 
s = tf.SparseTensor(indices=[[0, 1], [1, 0], [2, 3]], # index at which the non-zero element is at
                    values = [1., 2., 3.], # the value non-zero element
                    dense_shape=[3,4])  # shape of the tensor
print(s, tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64)) tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [5]:
try:
    s + 1
except Exception as e: 
    print(e)

unsupported operand type(s) for +: 'SparseTensor' and 'int'


In [6]:
tf.sparse.sparse_dense_matmul(
    s, 
    tf.constant([[10., 20.], [30., 40.], [50., 60.], [70., 80.]])
)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 30.,  40.],
       [ 20.,  40.],
       [210., 240.]], dtype=float32)>

In [9]:
s2 = tf.SparseTensor(indices=[[0, 2], [0, 1]], # this is out of order ([0,2] should come after [0, 1])
                     values=[1., 2.],
                     dense_shape=[3, 4])

try: 
    tf.sparse.to_dense(s2)
except Exception as e: 
    print(e)